# Mech Tools Evaluation

### Create the dataset

In [ ]:
!docker build -t etl:0.0.1 . && docker run etl:0.0.1

### Load the dataset

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option("display.precision", 2)
pd.set_option('display.max_columns', None)

In [2]:
dataset = pd.read_csv("dataset.csv")
str_cols = ("id", "currentAnswer", "title", "requestId", "prompt", "tool", "nonce", "vote")
for col in str_cols:
    dataset[col] = dataset[col].astype("string")
dataset.head()

,id,currentAnswer,title,requestId,prompt,tool,nonce,p_yes,p_no,confidence,info_utility,vote,win_probability
0,0x0094fa304017d5c2b355790e2976f769ea600492,No,Will the Hisense U8K be considered a top-tier ...,1429730407779530824523722231071959771311408049...,"With the given question ""Will the Hisense U8K ...",prediction-online,c6366b3f-eff5-4533-8dd9-d653b281b29d,0.60,0.40,0.8,0.5,Yes,0.60
1,0x0094fa304017d5c2b355790e2976f769ea600492,No,Will the Hisense U8K be considered a top-tier ...,1695055931594747475916883029584567955775422500...,"With the given question ""Will the Hisense U8K ...",prediction-online,1eed33a5-a3f0-41c4-beae-23e9022ffe22,0.60,0.40,0.8,0.7,Yes,0.60
2,0x0094fa304017d5c2b355790e2976f769ea600492,No,Will the Hisense U8K be considered a top-tier ...,5972945302788386668720465960403202339977906500...,"With the given question ""Will the Hisense U8K ...",prediction-online,dd376ef9-eb2c-4d9f-8a5a-cf9ae8deb0b3,0.60,0.40,0.8,0.7,Yes,0.60
3,0x0094fa304017d5c2b355790e2976f769ea600492,No,Will the Hisense U8K be considered a top-tier ...,1043402953919313937539182160739114840263108832...,"With the given question ""Will the Hisense U8K ...",prediction-online,91096f15-5e3b-4bf1-8178-f17f1efcf639,0.70,0.30,0.8,0.6,Yes,0.70
4,0x0094fa304017d5c2b355790e2976f769ea600492,No,Will the Hisense U8K be considered a top-tier ...,9433232780766388309643050548812272093999565778...,"With the given question ""Will the Hisense U8K ...",prediction-online,92321968-7888-4877-b33f-22fa4755fbc2,0.65,0.35,0.9,0.8,Yes,0.65


In [3]:
dataset.shape

(6987, 13)

In [4]:
dataset.describe()

,p_yes,p_no,confidence,info_utility,win_probability
count,6987.00,6987.00,6987.00,6987.00,6987.00
mean,0.49,0.51,0.79,0.58,0.69
std,0.21,0.21,0.08,0.18,0.09
min,0.00,0.00,0.50,0.00,0.53
25%,0.30,0.35,0.80,0.50,0.60
50%,0.60,0.40,0.80,0.60,0.70
75%,0.65,0.70,0.80,0.70,0.80
max,1.00,1.00,1.00,1.00,1.00


In [5]:
dataset["vote"].describe()

count     6987
unique       2
top        Yes
freq      4123
Name: vote, dtype: object

### Normalize confidences

In [6]:
dataset["confidence"].unique()

array([0.8 , 0.9 , 0.7 , 0.85, 0.6 , 0.5 , 1.  , 0.75, 0.99, 0.95])

In [7]:
dataset.loc[(dataset["confidence"] >= 0.9) & (dataset["confidence"] < 1), "confidence"] = 0.9
dataset.loc[dataset["confidence"] == 0.85, "confidence"] = 0.8
dataset.loc[dataset["confidence"] == 0.75, "confidence"] = 0.7
dataset["confidence"].unique()

array([0.8, 0.9, 0.7, 0.6, 0.5, 1. ])

### Check the percentage of wins vs confidence for all the tools

In [8]:
def accuracy(data):
    correct_answers_mask = data["currentAnswer"] == data["vote"]
    n_answers = correct_answers_mask.count()
    n_answers_success = correct_answers_mask.sum()

    if n_answers == 0:
        accuracy = None
    elif n_answers_success == 0:
        accuracy = 0
    else:
        accuracy = n_answers_success/n_answers * 100

    return pd.Series({"n_correct": n_answers_success, "n_pred": n_answers, "accuracy": accuracy})

In [9]:
def acc_per_tool(group, col: str, conf: float):
    return group.apply(lambda x: accuracy(x[x[col] == conf]))

In [10]:
def gen_stats(group, col: str):
    stats = {f"{col}_{prob}" : acc_per_tool(group, col, prob) for prob in sorted(dataset[col].unique())}
    stats["total"] = tools_group.apply(accuracy)
    return pd.concat(stats.values(), axis=1, keys=stats.keys())

In [11]:
tools_group = dataset.groupby("tool")
tools_stats_per_conf = gen_stats(tools_group, "confidence")
display(tools_stats_per_conf)

confidence_0.5                  \
                                                n_correct n_pred accuracy   
tool                                                                        
claude-prediction-offline                            81.0  143.0    56.64   
claude-prediction-online                             26.0   37.0    70.27   
prediction-offline                                    0.0    0.0      NaN   
prediction-offline-sme                                0.0    0.0      NaN   
prediction-online                                     0.0    0.0      NaN   
prediction-online-sme                                 1.0    1.0   100.00   
prediction-sentence-embedding-bold                    0.0    0.0      NaN   
prediction-sentence-embedding-conservative            0.0    0.0      NaN   

                                           confidence_0.6                  \
                                                n_correct n_pred accuracy   
tool                                                                        
claude-prediction-offline                            12.0   19.0    63.16   
claude-prediction-online                             60.0   88.0    68.18   
prediction-offline                                    0.0    0.0      NaN   
prediction-offline-sme                               12.0   16.0    75.00   
prediction-online                                    33.0   40.0    82.50   
prediction-online-sme                                 2.0    2.0   100.00   
prediction-sentence-embedding-bold                    0.0    0.0      NaN   
prediction-sentence-embedding-conservative            0.0    0.0      NaN   

                                           confidence_0.7                  \
                                                n_correct n_pred accuracy   
tool                                                                        
claude-prediction-offline                            34.0   64.0    53.12   
claude-prediction-online                             85.0  144.0    59.03   
prediction-offline                                   14.0   19.0    73.68   
prediction-offline-sme                               72.0   87.0    82.76   
prediction-online                                   434.0  561.0    77.36   
prediction-online-sme                                36.0   47.0    76.60   
prediction-sentence-embedding-bold                   11.0   17.0    64.71   
prediction-sentence-embedding-conservative            0.0    0.0      NaN   

                                           confidence_0.8                   \
                                                n_correct  n_pred accuracy   
tool                                                                         
claude-prediction-offline                            30.0    44.0    68.18   
claude-prediction-online                             11.0    26.0    42.31   
prediction-offline                                   32.0    75.0    42.67   
prediction-offline-sme                              131.0   296.0    44.26   
prediction-online                                  1651.0  4118.0    40.09   
prediction-online-sme                               114.0   209.0    54.55   
prediction-sentence-embedding-bold                    8.0     8.0   100.00   
prediction-sentence-embedding-conservative            2.0     6.0    33.33   

                                           confidence_0.9                  \
                                                n_correct n_pred accuracy   
tool                                                                        
claude-prediction-offline                            30.0   37.0    81.08   
claude-prediction-online                              1.0    1.0   100.00   
prediction-offline                                    5.0   11.0    45.45   
prediction-offline-sme                               13.0   32.0    40.62   
prediction-online                                   372.0  748.0    49.73   
prediction-online-sme   

### Check the percentage of wins vs probability for all the tools

In [12]:
tools_stats_per_prob = gen_stats(tools_group, "win_probability")
tools_stats_per_prob

win_probability_0.525         \
                                                       n_correct n_pred   
tool                                                                      
claude-prediction-offline                                    0.0    0.0   
claude-prediction-online                                     0.0    0.0   
prediction-offline                                           0.0    0.0   
prediction-offline-sme                                       0.0    0.0   
prediction-online                                            0.0    0.0   
prediction-online-sme                                        0.0    1.0   
prediction-sentence-embedding-bold                           0.0    0.0   
prediction-sentence-embedding-conservative                   0.0    0.0   

                                                    win_probability_0.55  \
                                           accuracy            n_correct   
tool                                                                       
claude-prediction-offline                       NaN                  0.0   
claude-prediction-online                        NaN                  2.0   
prediction-offline                              NaN                  1.0   
prediction-offline-sme                          NaN                  1.0   
prediction-online                               NaN                 16.0   
prediction-online-sme                           0.0                  0.0   
prediction-sentence-embedding-bold              NaN                  0.0   
prediction-sentence-embedding-conservative      NaN                  0.0   

                                                            \
                                           n_pred accuracy   
tool                                                         
claude-prediction-offline                     0.0      NaN   
claude-prediction-online                      3.0    66.67   
prediction-offline                            2.0    50.00   
prediction-offline-sme                        1.0   100.00   
prediction-online                            26.0    61.54   
prediction-online-sme                         1.0     0.00   
prediction-sentence-embedding-bold            0.0      NaN   
prediction-sentence-embedding-conservative    0.0      NaN   

                                           win_probability_0.6          \
                                                     n_correct  n_pred   
tool                                                                     
claude-prediction-offline                                 44.0    76.0   
claude-prediction-online                                  77.0   138.0   
prediction-offline                                        17.0    46.0   
prediction-offline-sme                                    56.0   150.0   
prediction-online                                        712.0  2312.0   
prediction-online-sme                                     48.0   106.0   
prediction-sentence-embedding-bold                        11.0    17.0   
prediction-sentence-embedding-conservative                 2.0     5.0   

                                                    win_probability_0.65  \
                                           accuracy            n_correct   
tool                                                                       
claude-prediction-offline                     57.89                  0.0   
claude-prediction-online                      55.80                  7.0   
prediction-offline                            36.96                  0.0   
prediction-offline-sme                        37.33                  3.0   
prediction-online                             30.80                 82.0   
prediction-online-sme                         45.28                  4.0   
prediction-sentence-embedding-bold            64.71                  0.0   
prediction-sentence-embedding-conservative    40.00                  0.0   

                                             

### List the well performing tools

In [13]:
# we are using 55% as the performance threshold
perf_threshold = 55

In [14]:
well_performing = tools_stats_per_conf.loc[:, ("total", "accuracy")] > perf_threshold
tools_stats_per_conf.loc[well_performing, ("total", "accuracy")]

tool
claude-prediction-offline             63.11
claude-prediction-online              61.82
prediction-online-sme                 56.03
prediction-sentence-embedding-bold    85.00
Name: (total, accuracy), dtype: float64